In [1]:
import numpy as np
import pandas as pd

In [2]:
labeling_gte9_dl_path = '/Users/salmadanu/Desktop/Skripsi/skripsi-env/narasipal/dl_models/labeling_gte9_dl.xlsx'
df_labeling_gte9_dl = pd.read_excel(labeling_gte9_dl_path)

merged_raw_path = '/Users/salmadanu/Desktop/Skripsi/skripsi-env/narasipal/cleaning/merged_raw.xlsx'
df_merged_raw = pd.read_excel(merged_raw_path)

In [3]:
# Strip whitespace from both DataFrames
df_labeling_gte9_dl['judul_berita'] = df_labeling_gte9_dl['judul_berita'].str.strip()
df_merged_raw['judul_berita'] = df_merged_raw['judul_berita'].str.strip()

In [4]:
import re

df_labeling_gte9_dl['judul_berita'] = df_labeling_gte9_dl['judul_berita'].str.replace(r'\s+', ' ', regex=True)
df_merged_raw['judul_berita'] = df_merged_raw['judul_berita'].str.replace(r'\s+', ' ', regex=True)

# Run from here

In [5]:
import unicodedata
import re

def normalize_title(title):
    if isinstance(title, str):
        # Normalize unicode and whitespace
        title = unicodedata.normalize("NFKC", title)
        title = title.replace('\xa0', ' ')
        title = re.sub(r'\s+', ' ', title)
        title = re.sub(r'[\u200B-\u200D\uFEFF]', '', title)
        title = title.strip()
        # Remove common prefixes (e.g., "VIDEO:", "[HOAKS]", etc.)
        title = re.sub(
            r'^(?i)(video[:\s-]*|[\[\(]?(hoaks|klarifikasi|infografik|full|populer(?:\s+\w+)?|eksklusif|[a-z]*grafik|update|live|info)[\]\)]?[:\s-]*|[a-z]*grafik[:\s-]*)',
            '',
            title
        )

        # Remove any remaining leading non-alphanumeric characters
        title = re.sub(r'^[^\w]*', '', title)

    return title


In [6]:
df_labeling_gte9_dl['judul_berita_clean'] = df_labeling_gte9_dl['judul_berita'].apply(normalize_title)
df_merged_raw['judul_berita_clean'] = df_merged_raw['judul_berita'].apply(normalize_title)

/var/folders/yz/d10v6srd14b8wkvpxfd2br140000gp/T/ipykernel_26184/624159780.py:13: DeprecationWarning: Flags not at the start of the expression '^(?i)(video[:\\s-]*|[' (truncated)
  title = re.sub(


In [7]:
df_labeled_final = pd.merge(
    df_merged_raw,
    df_labeling_gte9_dl[['judul_berita_clean', 'frame_labels']],
    on='judul_berita_clean',
    how='inner'
)

df_labeled_final = df_labeled_final[['judul_berita_clean', 'sumber_berita', 'frame_labels']]
df_labeled_final.drop_duplicates(inplace=True)

In [8]:
df_labeled_final.to_excel('/Users/salmadanu/Desktop/Skripsi/skripsi-env/narasipal/labeling/v5_df_labeled_final.xlsx', index=True)

In [9]:
# Clean both first (as shown above)

# Left join to see what didn't match from labeling data
unmatched = df_labeling_gte9_dl[~df_labeling_gte9_dl['judul_berita_clean'].isin(df_merged_raw['judul_berita_clean'])]

print("Unmatched judul_berita_clean count:", len(unmatched))

Unmatched judul_berita_clean count: 0


In [10]:
# Create sets for fast comparison
set_label = set(df_labeling_gte9_dl['judul_berita_clean'])
set_raw = set(df_merged_raw['judul_berita_clean'])

# What's in df_labeling_gte9_dl but not in df_merged_raw
unmatched_titles = list(set_label - set_raw)

In [11]:
unmatched_titles

[]

In [12]:
from rapidfuzz import process, fuzz

# Clean titles again if needed
cleaned_raw_titles = df_merged_raw['judul_berita_clean'].tolist()

# Get top fuzzy matches for each unmatched title
for title in unmatched_titles[:46]:  # sample first 10
    match, score, idx = process.extractOne(title, cleaned_raw_titles, scorer=fuzz.ratio)
    print(f"\n🔎 {title}\n→ Closest match: {match} (score: {score})")

In [13]:
# Step 1: Check for duplicates in your merge key
df_labeling_gte9_dl['judul_berita_clean'].duplicated().sum()
df_merged_raw['judul_berita_clean'].duplicated().sum()  # or whatever df you're merging on

# Step 2: Try a groupby to find which key caused multiple matches
df_labeled_final['judul_berita_clean'].value_counts().head(10)

judul_berita_clean
Momen Warga Israel Disandera Hamas saat Hadiri Festival Musik            4
Zulhas: Jokowi Minta AS Bantu Hentikan Serangan Israel ke Palestina      2
4 Warga Palestina Tewas Ditembak Tentara Israel di Tepi Barat            2
Hamas: Keputusan Hentikan Perang Gaza Ada di Tangan AS                   2
Jokowi Instruksikan Prabowo Kirim Nakes dan Bangun RS di Gaza            2
Indonesia Dorong Pemberian Hak Istimewa ke Palestina di Sidang PBB       2
Serangan Israel Tewaskan Eks Menteri Agama Palestina Youssef Salama      2
Momen Prabowo Jabat Erat Tangan Presiden Palestina Mahmoud Abbas         2
Momen Prabowo Cium Bendera Palestina Saat Kampanye di Batam              2
Israel Izinkan Muslim Palestina Ibadah di Masjid Al-Aqsa saat Ramadan    2
Name: count, dtype: int64